In [3]:
from torchtext.vocab import GloVe
embedding_glove = GloVe(name='6B', dim=50)

# Self-attention
Self-attention is a sequence-to-sequence operation: a sequence of vectors goes in, and a sequence of vectors comes out. Let’s call the input vectors 𝐱1,𝐱2,…𝐱t and the corresponding output vectors 𝐲1,𝐲2,…,𝐲t. The vectors all have dimension k.

To apply self-attention, we simply assign each word t in our vocabulary an embedding vector 𝐯t (the values of which we’ll learn). This is what’s known as an embedding layer in sequence modeling. It turns the word sequence the,cat,walks,on,the,street into the vector sequence

𝐯-the,𝐯-cat,𝐯-walks,𝐯-on,𝐯-the,𝐯-street.


In [4]:
import torch
import torch.nn.functional as F
X = torch.stack((embedding_glove['the'], embedding_glove['cat'], embedding_glove['walks'], embedding_glove['on'], embedding_glove['the'], embedding_glove['street']))
print(X.shape)
X = X.reshape(1, X.shape[0], X.shape[1])
print(X.shape)

torch.Size([6, 50])
torch.Size([1, 6, 50])


To produce output vector 𝐲i, the self attention operation simply takes a weighted average over all the input vectors

$$𝐲_{i}= \sum_{j}w_{ij}𝐱_{j}$$
Where j indexes over the whole sequence and the weights sum to one over all j. The weight wij is not a parameter, as in a normal neural net, but it is derived from a function over 𝐱i and 𝐱j. The simplest option for this function is the dot product:

$$w^′_{ij}=𝐱_{i}^T𝐱_{j}$$
Note that 𝐱i is the input vector at the same position as the current output vector 𝐲i. For the next output vector, we get an entirely new series of dot products, and a different weighted sum.
![Image](http://peterbloem.nl/files/transformers/self-attention.svg)


In [5]:
raw_weights = torch.bmm(X, X.transpose(1, 2))
#   torch.bmm is a batched matrix multiplication. It 
#   applies matrix multiplication over batches of 
#   matrices.

The dot product gives us a value anywhere between negative and positive infinity, so we apply a softmax to map the values to $[0,1]$ and to ensure that they sum to 1 over the whole sequence:

$$w_{ij}= \frac{exp(w^′_{ij})}{∑_{j}exp(w^′_{ij})}$$
And that’s the basic operation of self attention.
To turn the raw weights $w^′_{ij}$ into positive values that sum to one, we apply a row-wise softmax:

In [6]:
weights = F.softmax(raw_weights, dim=2)

To compute the output sequence, we just multiply the weight matrix by 𝐗. This results in a batch of output matrices 𝐘 of size (b, t, k) whose rows are weighted sums over the rows of 𝐗.
$$𝐲_{i}=∑_{j}w_{ij}𝐱_{j}$$

In [8]:
Y = torch.bmm(weights, X)

Y is the output-vector with self-attention.